## NOTE : This notebook was originally created by Dr. Brian Chapman and others.  It has been modified slightly for BMI 6018 2017 course with additional modifications for BMI6115 2018 course.

# Identifying Patient Cohorts in [MIMIC-II](http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3124312/)


[MIMIC-II](https://physionet.org/mimic2/mimic2_clinical_overview.shtml) is a freely available database of ICU patients. To access the full database (now migrated to [MIMIC-III](https://www.nature.com/articles/sdata201635.pdf))  you must sign a data use agreement. However, there is a [demo data set](https://physionet.org/mimic2/demo/) based on 4000 deceased patients that can be used without signing any DUA.

## How to Use the MIMIC-II Database
* [MIMIC-II Cookbook](https://physionet.org/mimic2/demo/MIMICIICookBook_v1.pdf)
* [MIMIC Data Dictionaries](http://physionet.incor.usp.br/physiobank/database/dictionaries/)


## The Varieties of...Data
The data set is very rich and so is a good resource for exploring the varieties of clinical data

![MIMIC Paper](./images/mimic_paper_header.jpg)
![MIMIC Publications](./images/mimic_publications.jpg)
(Sources : https://mimic.physionet.org/)

Data incluces free text notes (nursing, radiology, discharg summaries, etc.), input/output events, test results, procedure codes, diagnosis codes, etc.

# Very Short FAQ : 
* Q : What is the difference between MIMIC-II and MIMIC-III?
* A : MIMIC-II spans the time period of 2001 to 2008.  MIMIC-III spans 2001 to 2012 so it contains more data.  In addition, some data structures have been improved to make MIMIC-III easier to work with.  Some data quality issues have been resolved as well


* Q : How can I get access to MIMIC-III for my own research?
* A : For full MIMIC-III dataset. You'll need to do CITI training and then some other steps.  Start here: https://mimic.physionet.org/gettingstarted/access/
   However, if you only need fewer data for exercise or experiment, you can use [MIMIC demo dataset](https://physionet.org/mimic2/demo/) without requiring DUA (data use agreement) and CITI training. 

In [1]:
%matplotlib inline

In [2]:
import pymysql
import pandas as pd
import getpass
import pandas as pd
import seaborn as sns

In [4]:
conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()


Enter MySQL passwd for jovyan········


In [ ]:
cursor.execute("SELECT count(*) from noteevents limit 10")

In [ ]:
#tables = pd.read_sql("SELECT table_name FROM information_schema.tables where table_schema='mimic2'", conn)
print(tables)


## Example Query: Identifying ICD9 Codes for Patients

In [ ]:

icd9_codes = pd.read_sql('SELECT count(*) from noteevents   where text like "%breast cancer%" limit  10',conn)
print(icd9_codes)


In [ ]:
icd9_codes = pd.read_sql('SELECT * from icd9 limit 100',conn)

In [ ]:
icd9_counts = icd9_codes["description"].value_counts(["description"]).to_frame(name="ICD9 Counts")
icd9_counts.head(10)

## Selecting Cohorts

Our most interesting explorations will be when we use information from multiple tables to limit/select cases. Here is an example selecting radiology reports for patients with COPD.

## Select all the radiology reports for a patient with COPD
### [Codes obtained from https://www.findacode.com/](https://www.findacode.com/)
* chronic bronchitis (ICD-9 codes 490-491)
* emphysema (ICD-9 code 492)
* bronchiectasis (ICD-9 code 494)
* chronic airway obstruction (ICD-9 code 496). 

The **\** character indicates a line continuation.

In [ ]:
copd_data = \
pd.read_sql("""SELECT noteevents.subject_id, 
                      noteevents.category, 
                      noteevents.text, 
                      icd9.code 
               FROM noteevents INNER JOIN icd9 ON 
                      noteevents.subject_id = icd9.subject_id 
               WHERE (   icd9.code LIKE '490%' OR
                         icd9.code LIKE '491%' OR
                         icd9.code LIKE '492%' OR
                         icd9.code LIKE '494%' OR
                         icd9.code LIKE '496%'
                      ) 
                      AND noteevents.category = 'RADIOLOGY_REPORT'""",conn)
copd_data.head(20)

In [ ]:
pad_data = \
pd.read_sql("""SELECT distinct noteevents.subject_id,  
                      noteevents.text
               FROM noteevents  
               WHERE (noteevents.text LIKE '% abi%')
               """,conn)

In [ ]:
pad_data.subject_id, pad_data.text

In [ ]:
len(pad_data)

In [ ]:
pad_data.columns



In [ ]:

doc=pad_data.iloc[0].text
print(doc)

In [9]:
import re
import pandas as pd

pattern=re.compile(r"breast cancer")
for d in pattern.findall(doc):
    print(d)
    type(d)

In [10]:
outdf = pd.DataFrame(columns=['DocID', 'Term', 'Start', 'End', 'DocText'])
for did, row in pad_data.iterrows():
    doc=row['text']
    did=row['subject_id']
    for m in re.finditer(pattern, doc):
        #print(m.start(), m.end(), m.group(0))
        s=m.start()-100
        e=m.end()+100
        #print(did , doc[s:e])
        outdf=outdf.append({'DocID':did,'Term':m, 'Start':s, 'End':e, 'DocText':doc}, ignore_index=True)


NameError: name 'pad_data' is not defined

In [11]:
pad_data2 = \
pd.read_sql("""SELECT noteevents.subject_id,  
                      noteevents.text 
               FROM noteevents  
               WHERE (   noteevents.text LIKE '%ciwa%'
                      )  """,conn)
pad_data2.head(20)
#print(pad_data2['text'])


,subject_id,text
0,78,\n\nAdmission Date: [**2778-3-24**] Dis...
1,78,\nMICU ADMISSION NOTE\nplease see fhpa for det...
2,590,\nMICU nursing Progress Note\nNeuro: Pt is res...
3,590,\nP-MICU NURSING NOTE\nS-VENTED.\nO-PLEASE SEE...
4,590,\nSMICU nsg progress\nNeuro- Alert and oriente...
5,773,\nMICU nursing admission note 5AM\nPt is 57 yo...
6,1194,\n \n \n \nAdmission Date: [**3489-1-23**] ...
7,1194,\n\nCCU NPN: please see flowsheet for objectiv...
8,1194,\n\nNSG NOTE\n\nCV: INITIAL HR 90-107. PT STAR...
9,1245,\n\nNPN (0700-1930) Review of Systems:\n\nNeur...


In [13]:
outdf2 = pd.DataFrame(columns=['DocID', 'Term', 'Start', 'End', 'DocText'])
for did, row in pad_data2.iterrows():
    doc=row['text']
    did=row['subject_id']
    for m in re.finditer(r'ciwa\b', doc):
        #print(m.start(), m.end(), m.group(0))
        s=m.start()-10
        e=m.end()+80
        outdf2=outdf2.append({'DocID':did,'Term':m, 'Start':s, 'End':e, 'DocText':doc}, ignore_index=True)


In [14]:
for index, row in outdf2.iterrows():
    start_snippet=int(row[2])    
    end_snippet=int(row[3])
    
    print(int(row[0]), row[1].group(0), ' " ', (row[4][start_snippet:end_snippet]).replace( '\n', ' '), ' ')

78 ciwa  "  prompted. ciwa scale for etoh/opiate w/d. patient appropriate, cooperative. awating urine tox   
78 ciwa  "  idine for ciwa/hr bp parameters, to give  if remains hypertensive and appears to be w/d ing. c  
1571 ciwa  "  ers /day) ciwa scale 4 at 2000 pt requesting something for [**Doctor Last Name 65**] ativan .5  
1571 ciwa  "   at 2100. ciwa scale for rest of shift 0.  Resp: 02 sat 92-96% on ra. ls clear. pcxr neg  cv:   
1571 ciwa  "  tivan per ciwa scale and agitation   
2114 ciwa  "  ng pt per ciwa scale.  A/P: Neuro sleepy despite no sedation.Aspiration precautions maintained  
2114 ciwa  "  following ciwa scale.   
2114 ciwa  "  following ciwa scale Med cautiously w ativan as needed. Close observation & possible 1:1 sitte  
3424 ciwa  "  lium, and ciwa scales. Avg req. 10mg valium q 1/2 hour, and 8mg ativan gtt per/h. Now w/ impro  
4512 ciwa  "  eruo: see ciwa scale pt atimes is agitated fidgeting, oriented x1 and does follows simple comm  
4512 ciwa  "  doses per c

## Exercise

* Based on the query described on page 19 of the [MIMIC-II Cookbook]( https://physionet.org/mimic2/demo/MIMICIICookBook_v1.pdf ) create a dataframe of body temperature from the database. Limit the query to a reasonable number of results
* Create a visualization of the values

## Exercise 

Come up with a visualization of the top ICD9 codes

## Exercise

If you do not know details about your data base, how might you use Pandas to discover the nature of your database? For exmaple, how might you learn the possible values for ``category`` in ``chart_events``?

## Exercise

1. Use online resources (e.g. [findacode](https://www.findacode.com/search/search.php), [CMS](https://www.cms.gov/medicare/coding/ICD9providerdiagnosticcodes/codes.html)) or clinical knowledge to select patients with a disease (or diseases) of interest to you. Use the mimic cookbook or data dictionaries to identify variables of interest.

<br/><br/>This material presented as part of the DeCART Data Science for the Health Science Summer Program at the University of Utah in 2017.<br/>
Presenters : Dr. Wendy Chapman, Jianlin Shi and Kelly Peterson

In [ ]:
conn2 = pymysql.connect(host="mysql.chpc.utah.edu",
                       port=3306,user=getpass.getpass("Enter MySQL MIMIC III user name:"),
                       passwd=getpass.getpass("Enter MySQL password for MIMICIII"),db='mimic3')
cursor2 = conn2.cursor()


In [ ]:
tables = pd.read_sql("SELECT table_name FROM information_schema.tables where table_schema  like '%mimic3%';", conn2)
print(tables)

In [ ]:
docs  = pd.read_sql("select count(*) from mimic3.NOTEEVENTS limit 10", conn2)
print(docs)

In [ ]:
icd9_codes = pd.read_sql('SELECT * from noteevents where text like "%fever%" limit 10',conn)
rows=[]
for id,row in icd9_codes.iterrows():
    rows.append(row['text'])

In [ ]:
rows[3]